# Great Circle Problem

**author**: *aleksander.grm@fpp.uni-lj.si*,<br>
**date**: 2024

In the analysis of a great circle voyage, we fundamentally distinguish **two** problems:
- determination of the distance $D$, departure course $\omega_1$, and vertex position $P_v$
- determination of positions of waypoints $P_m$ for navigation along intermediate rhumb lines

Let's examine the calculation of parameters for both problems. In the first case, we determine all parameters of the great circle and one intermediate point. In the second case, we calculate all intermediate points and determine the distance along rhumb lines. For both cases, we also plot the course of the voyage.

We enter positions in the format commonly used in navigation:

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$,

where degrees are given as whole numbers and minutes to one decimal place precision.

Input format for position:
- LAT-$\varphi$ and LONG-$\lambda$ are entered in the format [degrees, minutes, cardinal direction]

Example input for a position:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- The cardinal direction must be in **single quotes**, with permissible characters 'N', 'S', 'E', 'W'

In [ ]:
# Nedeed modules
import os, sys

# nav tools path
sys.path.append('../nav_tools')

In [ ]:
import numpy as np
import navtools as nt
import rhumbline as rl
import greatcircle as gc

### Case 1

In [ ]:
# Departure
fi0 = [45,33.6,'N']
la0 = [25,47.7,'E']
P0 = [nt.nav2dd(fi0),nt.nav2dd(la0)]

# Arrival
fi1 = [37,17.8,'N']
la1 = [102,27.4,'W']
P1 = [nt.nav2dd(fi1),nt.nav2dd(la1)]

# GC calculation
[d_gc, w1_gc, Pv] = gc.getGCparameters(P0, P1)

# Print results
print('departure:',nt.getPositionString(P0))
print('  arrival:',nt.getPositionString(P1))
print('----------------')
print('distance: {:.1f}Nm'.format(d_gc))
print('  course: {:.2f}{:s}'.format(w1_gc,nt.arc_deg))
print('  vertex:',nt.getPositionString(Pv))

In [ ]:
# Calculation of a midpoint
la_md = nt.nav2dd([180,0,'W'])
fi_md = gc.getMidPosition(P0,P1,la_md)
print('midpoint:', nt.getPositionString([fi_md, la_md]))

In [ ]:
# Draw result
P_gc = gc.getPathPoints(P0, P1)
nt.plotGCPath(P_gc,Pv,5,10,'gc_path_01.pdf')

In [ ]:
# Draw CG using RL parts
dla = 10 # GC dissection to 10 deg in longitude
P_gc = gc.getPathPoints(P0, P1, dla)
nt.plotGCRLPath(P_gc,10,20,'gc_RL-path_01.pdf')

print('p[{:02d}]: {:s}'.format(0, nt.getPositionString(P_gc[0])))
d_sum = 0.0
for i in range(1,P_gc.shape[0]):
    [d, w] = rl.rhumbLineP1(P_gc[i-1], P_gc[i])
    d_sum += d
    print('p[{:02d}]: {:s}; ({:02d}->{:02d}) {:s}={:.1f}{:s}; d={:.1f}Nm'.
          format(i, nt.getPositionString(P_gc[i]), i-1, i, nt.u_omega, w, nt.arc_deg,d))

print()    
print('distance-RL: {:.1f}Nm; diff: {:.1f}Nm'.format(d_sum, d_sum-d_gc))

### Case 2

In [ ]:
# Departure
fi0 = [27,12.5,'S']
la0 = [153,35.7,'E']
P0 = [nt.nav2dd(fi0),nt.nav2dd(la0)]

# Arrival
fi1 = [48,29.4,'N']
la1 = [124,48.6,'W']
P1 = [nt.nav2dd(fi1),nt.nav2dd(la1)]

# GC calculation
[d_gc, w1_gc, Pv] = gc.getGCparameters(P0, P1)

# Print results
print('departure:',nt.getPositionString(P0))
print('  arrival:',nt.getPositionString(P1))
print('----------------')
print('distance: {:.1f}Nm'.format(d_gc))
print('  course: {:.2f}{:s}'.format(w1_gc,nt.arc_deg))
print('  vertex:',nt.getPositionString(Pv))

In [ ]:
# Midpoint calculation
la_md = nt.nav2dd([180,0,'W'])
fi_md = gc.getMidPosition(P0,P1,la_md)
print('midpoint:', nt.getPositionString([fi_md, la_md]))

In [ ]:
# Draw result
P_gc = gc.getPathPoints(P0, P1)
nt.plotGCPath(P_gc,Pv,5,10,'gc_path_02.pdf')

In [ ]:
# Draw CG using RL parts
dla = 10 # GC dissection to 10 deg in longitude
P_gc = gc.getPathPoints(P0, P1, dla)
nt.plotGCRLPath(P_gc,10,20,'gc_RL-path_02.pdf')

print('p[{:02d}]: {:s}'.format(0, nt.getPositionString(P_gc[0])))
d_sum = 0.0
for i in range(1,P_gc.shape[0]):
    [d, w] = rl.rhumbLineP1(P_gc[i-1], P_gc[i])
    d_sum += d
    print('p[{:02d}]: {:s}; ({:02d}->{:02d}) {:s}={:.1f}{:s}; d={:.1f}Nm'.
          format(i, nt.getPositionString(P_gc[i]), i-1, i, nt.u_omega, w, nt.arc_deg,d))

print()    
print('distance-RL: {:.1f}Nm; diff: {:.1f}Nm'.format(d_sum, d_sum-d_gc))

### Case 3

In [ ]:
# Departure
fi0 = [50,55,'N']
la0 = [7,32,'W']
P0 = [nt.nav2dd(fi0),nt.nav2dd(la0)]

# Arrival
fi1 = [45,45,'N']
la1 = [53,24,'W']
P1 = [nt.nav2dd(fi1),nt.nav2dd(la1)]

# GC calculation
[d_gc, w1_gc, Pv] = gc.getGCparameters(P0, P1)

# Print results
print('departure:',nt.getPositionString(P0))
print('  arrival:',nt.getPositionString(P1))
print('----------------')
print('distance: {:.1f}Nm'.format(d_gc))
print('  course: {:.2f}{:s}'.format(w1_gc,nt.arc_deg))
print('  vertex:',nt.getPositionString(Pv))

In [ ]:
# Midpoint calculation
la_md = nt.nav2dd([45,34.2,'W'])
fi_md = gc.getMidPosition(P0,P1,la_md)
print('midpoint:', nt.getPositionString([fi_md, la_md]))

In [ ]:
# Draw result
P_gc = gc.getPathPoints(P0, P1)
nt.plotGCPath(P_gc,Pv,5,10,'gc_path_03.pdf')

In [ ]:
# Draw CG using RL parts
dla = 10 # GC dissection to 10 deg in longitude
P_gc = gc.getPathPoints(P0, P1, dla)
nt.plotGCRLPath(P_gc,5,10,'gc_RL-path_03.pdf')

print('p[{:02d}]: {:s}'.format(0, nt.getPositionString(P_gc[0])))
d_sum = 0.0
for i in range(1,P_gc.shape[0]):
    [d, w] = rl.rhumbLineP1(P_gc[i-1], P_gc[i])
    d_sum += d
    print('p[{:02d}]: {:s}; ({:02d}->{:02d}) {:s}={:.1f}{:s}; d={:.1f}Nm'.
          format(i, nt.getPositionString(P_gc[i]), i-1, i, nt.u_omega, w, nt.arc_deg,d))

print()    
print('distance-RL: {:.1f}Nm; diff: {:.1f}Nm'.format(d_sum, d_sum-d_gc))

### Case 4

Check for the Equator crossing.

In [ ]:
# Položaj odhoda
fi0 = [32,33.6,'N']
la0 = [13,34.8,'W']
P0 = [nt.nav2dd(fi0),nt.nav2dd(la0)]

# Položaj prihoda
fi1 = [63,24.5,'S']
la1 = [72,22.3,'W']
P1 = [nt.nav2dd(fi1),nt.nav2dd(la1)]

# GC calculation
[d_gc, w1_gc, Pv] = gc.getGCparameters(P0, P1)

# Print results
print('departure:',nt.getPositionString(P0))
print('  arrival:',nt.getPositionString(P1))
print('----------------')
print('distance: {:.1f}Nm'.format(d_gc))
print('  course: {:.2f}{:s}'.format(w1_gc,nt.arc_deg))
print('  vertex:',nt.getPositionString(Pv))

In [ ]:
# Midpoint calculation
la_md = nt.nav2dd([45,34.2,'W'])
fi_md = gc.getMidPosition(P0,P1,la_md)
print('midpoint:', nt.getPositionString([fi_md, la_md]))

In [ ]:
# Draw result
P_gc = gc.getPathPoints(P0, P1)
nt.plotGCPath(P_gc,Pv,10,20,'gc_path_04.pdf')

In [ ]:
# Draw CG using RL parts
dla = 10 # GC dissection to 10 deg in longitude
P_gc = gc.getPathPoints(P0, P1, dla)
nt.plotGCRLPath(P_gc,10,20,'gc_RL-path_04.pdf')

print('p[{:02d}]: {:s}'.format(0, nt.getPositionString(P_gc[0])))
d_sum = 0.0
for i in range(1,P_gc.shape[0]):
    [d, w] = rl.rhumbLineP1(P_gc[i-1], P_gc[i])
    d_sum += d
    print('p[{:02d}]: {:s}; ({:02d}->{:02d}) {:s}={:.1f}{:s}; d={:.1f}Nm'.
          format(i, nt.getPositionString(P_gc[i]), i-1, i, nt.u_omega, w, nt.arc_deg,d))

print()    
print('distance-RL: {:.1f}Nm; diff: {:.1f}Nm'.format(d_sum, d_sum-d_gc))